# Recurrent models 101

In the evolving landscape of Natural Language Processing (NLP), Recurrent Neural Networks (RNNs) stand out for their unique ability to process sequential data, making them particularly suited for text analysis tasks. This practical exercise is designed to provide hands-on experience with RNNs, focusing on their application in classifying product reviews as either positive or negative.

Product reviews, ubiquitous across online platforms, serve as valuable data for extracting consumer sentiments and preferences. However, the unstructured nature of text data poses a challenge for traditional machine learning models. RNNs, with their sequential data processing capability, offer a robust solution to this challenge. By considering the temporal dynamics of language, RNNs can capture contextual nuances critical for accurately interpreting sentiments.

This practical will guide you through the end-to-end process of leveraging RNNs for sentiment analysis, encompassing the following steps:

- Reading the Dataset: We begin by importing a dataset comprising product reviews, each tagged with additional information such as review summary, verification status of the purchase, timestamp, adjusted log votes, and a binary indicator of the review's sentiment (positive or negative).
- Exploratory Data Analysis (EDA): Before delving into modeling, we will conduct a preliminary analysis to understand the dataset's characteristics and distribution.
- Dataset Splitting: The dataset will be divided into training and validation sets to evaluate the model's performance.
- Text Processing and Transformation: We will preprocess the text data, converting it into a format suitable for RNN processing.
- Data Batching and Iterator Creation: This step involves creating batches of data and iterators for efficiently feeding data into the model during training.
- Utilizing Pre-trained GloVe Word Embeddings: To enhance the model's understanding of language semantics, we will use GloVe (Global Vectors for Word Representation), a pre-trained word embedding.
- Setting Hyperparameters and Building the Network: We will configure the RNN's hyperparameters and architecture.
- Training the Model: The network will be trained on the prepared dataset.
- Validation: The trained model's performance will be evaluated on the validation dataset.
- Improvement Ideas: We will explore strategies for further enhancing the model's accuracy.


An important distinction to note is that while RNNs excel in processing textual data, incorporating additional features such as timestamps, log_votes, and verification status requires integrating RNNs with traditional neural network models. This practical will thus not only familiarize you with RNNs but also illustrate how to combine them with other neural network architectures to leverage both sequential and non-sequential data for comprehensive analysis.


In this practical you need to use torchtext

Import torchtext, GloVe vocabulary and get_tokenizer function from torchtext

In [15]:
import torch, torchtext
from torchtext.vocab import GloVe
from torchtext.data.utils import get_tokenizer

from collections import Counter

In [3]:
import time
import numpy as np
import torch, torchtext
import pandas as pd
from torch import nn, optim
from torch.nn import BCEWithLogitsLoss
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

You can download the dataset using this link: https://drive.google.com/file/d/1k1XD0XPRSCC4tGGX1LUN-uzrmlQNJjs2/view?usp=sharing

Then you can upload the dataset on your drive or directly in colab.

Now, Let's read the dataset:

In [4]:
import pandas as pd

df = pd.read_csv('./AMAZON-REVIEW-DATA-CLASSIFICATION.csv')

Display some information

In [ ]:
# FIXME

### Data analysis

Display some examples from the dataset

In [ ]:
# FIXME

Display the number of samples

In [ ]:
# FIXME

Display the label proportion (number of positive labels / negative labels)

In [ ]:
# FIXME

Display the value count for verified

In [ ]:
# FIXME

Remove NaN values

In [ ]:
# FIXME

### Split the dataset into training and validation

In [ ]:
# FIXME

### Text processing and Transformation

Use the get tokenizer function from torchtext to download `basic_english` tokenizer

In [ ]:
# FIXME

Use Counter() from collection, that we will use to compute the histogram of our tokens

In [ ]:
# FIXME

Update the counter for each samples of your dataset

`counter.update(tokenizer("..."))`

In [17]:
for line in train_text:
    # FIXME

Let's display the counter. You can see that we have the count for each token

In [ ]:
# FIXME

Create a vocabulary with words seen at least 5 (min_freq) times

use torchtext vocab to create your vocabulary

In [19]:
# FIXME

Add the unknow token and the pad token, using `insert_token` function

In [ ]:
# Add the unknown token
# and use it by default for unknown words
unk_token = '<unk>'
vocab.insert_token(unk_token, 0)
vocab.set_default_index(0)

# Add the pad token
pad_token = '<pad>'
vocab.insert_token(pad_token, 1)

Here you have some examples

In [ ]:
print(f"'home' -> {vocab['home']}")
print(f"'wash' -> {vocab['wash']}")
# unknown word (assume from test set)
print(f"'fhshbasdhb' -> {vocab['fhshbasdhb']}")

In [ ]:
# you can use the following mapper to tokenize the data
text_transform_pipeline = lambda x: [vocab[token] for token in tokenizer(x)]

In [ ]:
print(f"Before transform:\t{train_text[37]}")
print(f"After transform:\t{text_transform_pipeline(train_text[37])}")

Let's write a function that modifies and pads our text data as needed. In this function, we'll truncate the text sequence when it exceeds a specified length (in this case, max_len=50). If the text is shorter than max_len, we'll append 1s to the end of the sequence, representing the padding token.

In [ ]:
def transformText(text_list, max_len):
    # Transform the text
    transformed_data = [text_transform_pipeline(text)[:max_len] for text in text_list]

    # Pad zeros if the text is shoter than max_len
    # FIXME

In [ ]:
train_text[300]

In [ ]:
text = train_text[7:9]
print(f"Text: {text}\n")
print(f"Num sentences: {len(text)}\n")
tt = transformText(text, max_len=50)
print(f"Transformed text: \n{tt}\n")
print(f"Shape of transformed text: {tt.shape}")

Let's use the transformText() function and create the data loaders. Here, we use max_len=100 to consider the first 100 words in the text.

Use TensorDataset to build the dataset from you tokens and labels.

In [ ]:
max_len = 100
batch_size = 32

# Pass transformed and padded data to dataset
# Create data loaders
# FIXME

### Build the model

In this illustration, we'll leverage GloVe word vectors, specifically those from the '6B' dataset with dimensions of 300. This dataset encompasses 6 billion words and phrases, each represented by a vector containing 300 numerical values. The code below demonstrates how to access these word vectors and construct an embedding matrix. We'll establish a linkage between our vocabulary indices and the GloVe embeddings utilizing the get_vecs_by_tokens() function

In [ ]:
glove = GloVe(name="6B", dim=300)
embedding_matrix = # FIXME (use get vec by token)

Hyper parameters of the model that you can tune

In [ ]:
# Size of the state vectors
hidden_size = 8

# General NN training parameters
learning_rate = 0.001
epochs = 25

# Embedding vector and vocabulary sizes
embed_size = 300  # glove.6B.300d.txt
vocab_size = len(vocab.get_itos())

Before proceeding, it's essential to ensure our data is properly formatted for the subsequent stages. Our model comprises the following layers:

- Embedding Layer: Responsible for mapping words/tokens to word vectors.
- RNN Layer: In this instance, we employ a simple RNN model, consisting of two stacked RNN layers. Further insights into the RNN architecture can be found here.
- Linear Layer: Utilized for the final prediction of 'isPositive', this layer consists of a single neuron.

Embedding documentation: https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html

In [ ]:
class Net(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers=1):
        super().__init__()
        # FIXME

    def forward(self, inputs):
        # FIXME

# Initialize the weights
def init_weights(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
    if type(m) == nn.RNN:
        for param in m._flat_weights_names:
            if "weight" in param:
                nn.init.xavier_uniform_(m._parameters[param])


Let's initialize this network. Then, we will need to make the embedding layer use our GloVe word vectors.

Use `model.embedding.weight.data.copy_` to set the pretrained embedding parameters and freeze the weight

In [ ]:
# We set the embedding layer's parameters from GloVe
# FIXME

# We won't change/train the embedding layer, use requires_grad attributes
# FIXME

Build loss and optimizer

In [ ]:
# FIXME

Now build the train / validation loop

In [ ]:
# FIXME

### Test our classifier

Display some prediction and display the confusion matrix / accuracy / precision / recall / f1score on the validation set

In [ ]:
# FIXME

Improvement:

- Change the batch size
- Add more layers
- Use GRU / LSTM layers

In [ ]:
# FIXME